# PUDL FERC Form 6 Data for Yellowstone Pipe Line Company (YPL)

In [1]:
import sqlite3
import pandas as pd

## Purpose

The purpose of this notebook will be to explore how to extract Federal Energy Regulatory Commission [(FERC)](https://ferc.gov/what-ferc) data provided by the Public Utility Data Liberation [(PUDL)](https://catalyst.coop/pudl/) Project.

The goal will be to extract data for the Yellowstone Pipe Line Company from the PUDL Sqlite database and match to the FERC Form 6 exported from the FERC website.

## Establish Connection to PUDL Database

In [2]:
# Connect to the database
conn = sqlite3.connect("ferc6_xbrl.sqlite")

In [3]:
# Create a cursor that lets you execute SQL queries and fetch results
cursor = conn.cursor()

In [4]:
# Get list of table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

## Explore Tables Contained in the Database

In [5]:
# Assign list of tables to variable
tables = cursor.fetchall()

In [6]:
# Print list of table names
print("Tables in the database:")
for table in tables:
    print(table[0])

Tables in the database:
annual_corporate_officer_certification_001_duration
identification_001_duration
list_of_schedule_002_duration
general_information_101_duration
control_over_respondent_102_duration
control_over_respondent_102_instant
companies_controlled_by_respondent_103_duration
companies_controlled_by_respondent_103_instant
principal_general_officers_104_duration
directors_105_duration
important_changes_during_the_quarter_year_108_duration
comparative_balance_sheet_110_instant
income_statement_114_duration
statement_of_accumulated_other_comprehensive_income_and_hedging_activities_116_duration
statement_of_accumulated_other_comprehensive_income_and_hedging_activities_116_instant
appropriated_retained_income_118_instant
appropriated_retained_income_other_appropriations_118_duration
appropriated_retained_income_other_appropriations_118_instant
unappropriated_retained_income_statement_119_duration
unappropriated_retained_income_statement_119_instant
statement_of_cash_flows_120_dur

## Extract Income Statement Data

Below are screenshots of the FERC Form 6 Income Statement for Yellowstone Pipe Line Company:

![](img/ypl-income-statement-image-01.png)

![](img/ypl-income-statement-image-02.png)

![](img/ypl-income-statement-image-03.png)

![](img/ypl-income-statement-image-04.png)


By looking at the list of tables from earlier, "income_statement_114_duration" would appear to be the relevant table name as the Ferc Form 6, Income Statement is on Page 114 of the form. However, to determine if this is the correct table we will extract the table data into a Pandas DataFrame to be able to compare to the Income Statement screenshots above.

In [7]:
# Assign income statement table to a DataFrame then close the sql connection
df_is = pd.read_sql("SELECT * FROM income_statement_114_duration", conn)
conn.close()
print(df_is.head())

  entity_id                                        filing_name  \
0   C004254  Tallgrass_Pony_Express_Pipeline,_LLC_form6_Q4_...   
1   C004254  Tallgrass_Pony_Express_Pipeline,_LLC_form6_Q4_...   
2   C000954             Frontier_Aspen_LLC_form6_Q4_1649970270   
3   C000954             Frontier_Aspen_LLC_form6_Q4_1649970270   
4   C000070         NuStar_Logistics,_L.P._form6_Q4_1649102002   

             publication_time  start_date    end_date  \
0  2022-04-15 17:34:15.000000  2020-01-01  2020-12-31   
1  2022-04-15 17:34:15.000000  2021-01-01  2021-12-31   
2  2022-04-14 21:04:30.000000  2020-01-01  2020-12-31   
3  2022-04-14 21:04:30.000000  2021-01-01  2021-12-31   
4  2022-04-04 19:53:22.000000  2020-01-01  2020-12-31   

   provision_for_deferred_taxes_extraordinary_items  extraordinary_items_net  \
0                                               NaN                      NaN   
1                                               NaN                      NaN   
2                   

In [8]:
df_is.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2369 entries, 0 to 2368
Data columns (total 34 columns):
 #   Column                                                                              Non-Null Count  Dtype  
---  ------                                                                              --------------  -----  
 0   entity_id                                                                           2369 non-null   object 
 1   filing_name                                                                         2369 non-null   object 
 2   publication_time                                                                    2369 non-null   object 
 3   start_date                                                                          2369 non-null   object 
 4   end_date                                                                            2369 non-null   object 
 5   provision_for_deferred_taxes_extraordinary_items                                    312 non-null 

The initial DataFrame contains information for 2,000+ filers. For this example, the goal is to pull information for Yellowstone Pipe Line Company. To be able to do this the DataFrame will need to be filtered. Looking at the above it appears column **entity_id** contains a unique identifier for each filer and **filing_name** contains the name of the filer. The **filing_name** column will first be used to determine which **entity_id** corresponds to Yellowstone Pipe Line Company.

In [9]:
# Use str.contains to find Yellowstone Pipe Line's entity id
df_is[df_is.loc[:, 'filing_name'].str.contains('Yellowstone')]

,entity_id,filing_name,publication_time,start_date,end_date,provision_for_deferred_taxes_extraordinary_items,extraordinary_items_net,equity_in_earnings_losses_of_affiliated_companies_including_dividend_income,net_income_loss,income_taxes_on_extraordinary_items,...,federal_income_taxes_on_income_from_continuing_operations,net_carrier_operating_income,miscellaneous_income_charges,income_net_from_noncarrier_property,unusual_or_infrequent_items_debit,operating_revenues,other_income_and_deductions,interest_and_dividend_income,extraordinary_items,income_loss_before_extraordinary_items
326,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1650134001,2022-04-16 18:33:21.000000,2021-01-01,2021-12-31,NaN,NaN,NaN,10446634.0,NaN,...,4154368.0,14737038.0,6277.0,NaN,NaN,41152800.0,-2861.0,7631.0,NaN,10446634.0
327,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1650134001,2022-04-16 18:33:21.000000,2020-01-01,2020-12-31,NaN,NaN,NaN,8390951.0,NaN,...,2754429.0,11624247.0,7.0,NaN,NaN,38957812.0,79026.0,77372.0,NaN,8390951.0
638,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1681324161,2023-04-12 18:29:21.000000,2022-01-01,2022-12-31,NaN,NaN,NaN,10008065.0,NaN,...,4498345.0,14352988.0,21639.0,NaN,NaN,41400449.0,163597.0,195267.0,NaN,10008065.0
639,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1681324161,2023-04-12 18:29:21.000000,2021-01-01,2021-12-31,NaN,NaN,NaN,10446634.0,NaN,...,4154368.0,14737038.0,6277.0,NaN,NaN,41152800.0,-2861.0,7631.0,NaN,10446634.0
1460,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1712936573,2024-04-12 15:42:53.000000,2023-01-01,2023-12-31,NaN,NaN,NaN,14630430.0,NaN,...,4700520.0,19048477.0,4000.0,NaN,NaN,46690658.0,600111.0,601411.0,NaN,14630430.0
1461,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1712936573,2024-04-12 15:42:53.000000,2022-01-01,2022-12-31,NaN,NaN,NaN,10008065.0,NaN,...,4498345.0,14352988.0,21639.0,NaN,NaN,41400449.0,163597.0,195267.0,NaN,10008065.0
1837,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1745016653,2025-04-18 22:50:53.000000,2024-01-01,2024-12-31,NaN,NaN,NaN,17978665.0,NaN,...,7076805.0,24391749.0,6296.0,NaN,NaN,52343305.0,866826.0,825574.0,NaN,17978665.0
1838,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1745016653,2025-04-18 22:50:53.000000,2023-01-01,2023-12-31,NaN,NaN,NaN,14630430.0,NaN,...,4700520.0,19048477.0,4000.0,NaN,NaN,46690658.0,600111.0,601411.0,NaN,14630430.0


Yellowstone Pipe Line Company's entity id is **C001041**. We can now use this ID to filter the DataFrame for YPL.

In [10]:
# Use YPL's entity id to filter the DataFrame
id_ypl = 'C001041'
df_is_ypl = df_is[df_is['entity_id'] == id_ypl]

In [11]:
df_is_ypl

,entity_id,filing_name,publication_time,start_date,end_date,provision_for_deferred_taxes_extraordinary_items,extraordinary_items_net,equity_in_earnings_losses_of_affiliated_companies_including_dividend_income,net_income_loss,income_taxes_on_extraordinary_items,...,federal_income_taxes_on_income_from_continuing_operations,net_carrier_operating_income,miscellaneous_income_charges,income_net_from_noncarrier_property,unusual_or_infrequent_items_debit,operating_revenues,other_income_and_deductions,interest_and_dividend_income,extraordinary_items,income_loss_before_extraordinary_items
326,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1650134001,2022-04-16 18:33:21.000000,2021-01-01,2021-12-31,NaN,NaN,NaN,10446634.0,NaN,...,4154368.0,14737038.0,6277.0,NaN,NaN,41152800.0,-2861.0,7631.0,NaN,10446634.0
327,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1650134001,2022-04-16 18:33:21.000000,2020-01-01,2020-12-31,NaN,NaN,NaN,8390951.0,NaN,...,2754429.0,11624247.0,7.0,NaN,NaN,38957812.0,79026.0,77372.0,NaN,8390951.0
638,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1681324161,2023-04-12 18:29:21.000000,2022-01-01,2022-12-31,NaN,NaN,NaN,10008065.0,NaN,...,4498345.0,14352988.0,21639.0,NaN,NaN,41400449.0,163597.0,195267.0,NaN,10008065.0
639,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1681324161,2023-04-12 18:29:21.000000,2021-01-01,2021-12-31,NaN,NaN,NaN,10446634.0,NaN,...,4154368.0,14737038.0,6277.0,NaN,NaN,41152800.0,-2861.0,7631.0,NaN,10446634.0
1460,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1712936573,2024-04-12 15:42:53.000000,2023-01-01,2023-12-31,NaN,NaN,NaN,14630430.0,NaN,...,4700520.0,19048477.0,4000.0,NaN,NaN,46690658.0,600111.0,601411.0,NaN,14630430.0
1461,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1712936573,2024-04-12 15:42:53.000000,2022-01-01,2022-12-31,NaN,NaN,NaN,10008065.0,NaN,...,4498345.0,14352988.0,21639.0,NaN,NaN,41400449.0,163597.0,195267.0,NaN,10008065.0
1837,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1745016653,2025-04-18 22:50:53.000000,2024-01-01,2024-12-31,NaN,NaN,NaN,17978665.0,NaN,...,7076805.0,24391749.0,6296.0,NaN,NaN,52343305.0,866826.0,825574.0,NaN,17978665.0
1838,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1745016653,2025-04-18 22:50:53.000000,2023-01-01,2023-12-31,NaN,NaN,NaN,14630430.0,NaN,...,4700520.0,19048477.0,4000.0,NaN,NaN,46690658.0,600111.0,601411.0,NaN,14630430.0


From reviewing the above there will be overlap of data due to the fact that for each FERC form there is current and prior year data reported. This results in originally reported data being available (the current year data on each years form) and restated data being available (the prior year data on each years form). For this section of the analyis our goal is to match the current year FERC Form 6, so we will further filter the data to the most recent form. To do this we can filter using the **publication_time** column.

In [12]:
# Filter the DataFrame by publication_time
df_is_ypl_25 = df_is_ypl[df_is_ypl.loc[:, 'publication_time'].str.contains('2025-')]

In [13]:
df_is_ypl_25

,entity_id,filing_name,publication_time,start_date,end_date,provision_for_deferred_taxes_extraordinary_items,extraordinary_items_net,equity_in_earnings_losses_of_affiliated_companies_including_dividend_income,net_income_loss,income_taxes_on_extraordinary_items,...,federal_income_taxes_on_income_from_continuing_operations,net_carrier_operating_income,miscellaneous_income_charges,income_net_from_noncarrier_property,unusual_or_infrequent_items_debit,operating_revenues,other_income_and_deductions,interest_and_dividend_income,extraordinary_items,income_loss_before_extraordinary_items
1837,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1745016653,2025-04-18 22:50:53.000000,2024-01-01,2024-12-31,NaN,NaN,NaN,17978665.0,NaN,...,7076805.0,24391749.0,6296.0,NaN,NaN,52343305.0,866826.0,825574.0,NaN,17978665.0
1838,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1745016653,2025-04-18 22:50:53.000000,2023-01-01,2023-12-31,NaN,NaN,NaN,14630430.0,NaN,...,4700520.0,19048477.0,4000.0,NaN,NaN,46690658.0,600111.0,601411.0,NaN,14630430.0


From a quick review this does appear to be the correct information. To be sure we can put together a printout in an order that matches to the published FERC Form 6, Income Statement.

In [14]:
# Get a list of sorted column names
column_names = df_is_ypl_25.columns
column_names = column_names.sort_values()

In [15]:
# Print sorted list
for column in column_names:
    print(column)

cumulative_effect_of_changes_in_accounting_principles_less_applicable_income_taxes
dividend_income_equity_investments
end_date
entity_id
equity_in_earnings_losses_of_affiliated_companies_including_dividend_income
extraordinary_items
extraordinary_items_and_accounting_changes
extraordinary_items_net
federal_income_taxes_on_income_from_continuing_operations
filing_name
gain_loss_from_disposition_of_discontinued_segments_less_applicable_income_taxes
income_loss_before_extraordinary_items
income_loss_from_continuing_operations
income_loss_from_discontinued_operations
income_loss_from_operations_of_discontinued_segments_less_applicable_income_taxes
income_net_from_noncarrier_property
income_taxes_on_extraordinary_items
interest_and_dividend_income
interest_expense
miscellaneous_income
miscellaneous_income_charges
net_carrier_operating_income
net_income_loss
operating_expenses
operating_revenues
ordinary_income_before_federal_income_taxes
other_income_and_deductions
provision_for_deferred_ta

In [16]:
# Produce a printout of the DataFrame in an order that matches to the Income Statement
df_is_ypl_25.loc[:, [
    'publication_time',
    'start_date',
    'end_date',
    'operating_revenues', 
    'operating_expenses',
    'net_carrier_operating_income',
    'other_income_and_deductions',
    'income_net_from_noncarrier_property',
    'interest_and_dividend_income',
    'miscellaneous_income',
    'unusual_or_infrequent_items_credit',
    'interest_expense',
    'miscellaneous_income_charges',
    'unusual_or_infrequent_items_debit',
    'dividend_income_equity_investments',
    'undistributed_earnings_losses',
    'equity_in_earnings_losses_of_affiliated_companies_including_dividend_income',
    'other_income_and_deductions',
    'ordinary_income_before_federal_income_taxes',
    'federal_income_taxes_on_income_from_continuing_operations',
    'provision_for_deferred_taxes',
    'income_loss_from_continuing_operations',
    'income_loss_from_operations_of_discontinued_segments_less_applicable_income_taxes',
    'gain_loss_from_disposition_of_discontinued_segments_less_applicable_income_taxes',
    'income_loss_from_discontinued_operations',
    'income_loss_before_extraordinary_items',
    'extraordinary_items_net',
    'income_taxes_on_extraordinary_items',
    'provision_for_deferred_taxes_extraordinary_items',
    'extraordinary_items',
    'cumulative_effect_of_changes_in_accounting_principles_less_applicable_income_taxes',
    'extraordinary_items_and_accounting_changes',
    'net_income_loss'
    ]]

,publication_time,start_date,end_date,operating_revenues,operating_expenses,net_carrier_operating_income,other_income_and_deductions,income_net_from_noncarrier_property,interest_and_dividend_income,miscellaneous_income,...,gain_loss_from_disposition_of_discontinued_segments_less_applicable_income_taxes,income_loss_from_discontinued_operations,income_loss_before_extraordinary_items,extraordinary_items_net,income_taxes_on_extraordinary_items,provision_for_deferred_taxes_extraordinary_items,extraordinary_items,cumulative_effect_of_changes_in_accounting_principles_less_applicable_income_taxes,extraordinary_items_and_accounting_changes,net_income_loss
1837,2025-04-18 22:50:53.000000,2024-01-01,2024-12-31,52343305.0,27951556.0,24391749.0,866826.0,NaN,825574.0,50248.0,...,NaN,NaN,17978665.0,NaN,NaN,NaN,NaN,NaN,NaN,17978665.0
1838,2025-04-18 22:50:53.000000,2023-01-01,2023-12-31,46690658.0,27642181.0,19048477.0,600111.0,NaN,601411.0,2700.0,...,NaN,NaN,14630430.0,NaN,NaN,NaN,NaN,NaN,NaN,14630430.0


The DataFrame printout does materially match to the Income Statement screenshots taken from the FERC Form 6 for Yellowstone Pipe Line Company. For example, the first screenshot is again copied below:

![](img/ypl-income-statement-image-01.png)

## Income Statement Extract Function

We now can create a function that will extract the data for an **entity_id** in an order that matches to the FERC Form 6.

In [17]:
# Define the Income Statement function
def get_income_statement(db_file, sql_query, subject_id):

    # Connect to the sql database using sqlite3
    conn = sqlite3.connect(db_file)

    # Query the database file and assign to pandas dataframe
    df = pd.read_sql(sql_query + " WHERE entity_id = ?", conn, params=(subject_id,))
    
    # Close the database connection
    conn.close()

    # Rearange columns to match income statement format
    income_statement_items = (
        'entity_id',
        'filing_name',
        'publication_time',
        'start_date',
        'end_date',
        'operating_revenues', 
        'operating_expenses',
        'net_carrier_operating_income',
        'other_income_and_deductions',
        'income_net_from_noncarrier_property',
        'interest_and_dividend_income',
        'miscellaneous_income',
        'unusual_or_infrequent_items_credit',
        'interest_expense',
        'miscellaneous_income_charges',
        'unusual_or_infrequent_items_debit',
        'dividend_income_equity_investments',
        'undistributed_earnings_losses',
        'equity_in_earnings_losses_of_affiliated_companies_including_dividend_income',
        'other_income_and_deductions',
        'ordinary_income_before_federal_income_taxes',
        'federal_income_taxes_on_income_from_continuing_operations',
        'provision_for_deferred_taxes',
        'income_loss_from_continuing_operations',
        'income_loss_from_operations_of_discontinued_segments_less_applicable_income_taxes',
        'gain_loss_from_disposition_of_discontinued_segments_less_applicable_income_taxes',
        'income_loss_from_discontinued_operations',
        'income_loss_before_extraordinary_items',
        'extraordinary_items_net',
        'income_taxes_on_extraordinary_items',
        'provision_for_deferred_taxes_extraordinary_items',
        'extraordinary_items',
        'cumulative_effect_of_changes_in_accounting_principles_less_applicable_income_taxes',
        'extraordinary_items_and_accounting_changes',
        'net_income_loss'
    )
    
    df = df.loc[:, income_statement_items]
    
    return df

In [18]:
# Set function parameters
db_file = 'ferc6_xbrl.sqlite'
desired_table = 'income_statement_114_duration'
desired_columns = '*'
subject_id = 'C001041'
sql_query = f'SELECT {desired_columns} FROM {desired_table}'

In [19]:
# Query the PUDL SQL Database
df_ypl = get_income_statement(db_file, sql_query, subject_id)

In [20]:
df_ypl

,entity_id,filing_name,publication_time,start_date,end_date,operating_revenues,operating_expenses,net_carrier_operating_income,other_income_and_deductions,income_net_from_noncarrier_property,...,gain_loss_from_disposition_of_discontinued_segments_less_applicable_income_taxes,income_loss_from_discontinued_operations,income_loss_before_extraordinary_items,extraordinary_items_net,income_taxes_on_extraordinary_items,provision_for_deferred_taxes_extraordinary_items,extraordinary_items,cumulative_effect_of_changes_in_accounting_principles_less_applicable_income_taxes,extraordinary_items_and_accounting_changes,net_income_loss
0,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1650134001,2022-04-16 18:33:21.000000,2021-01-01,2021-12-31,41152800.0,26415762.0,14737038.0,-2861.0,None,...,None,None,10446634.0,None,None,None,None,None,None,10446634.0
1,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1650134001,2022-04-16 18:33:21.000000,2020-01-01,2020-12-31,38957812.0,27333565.0,11624247.0,79026.0,None,...,None,None,8390951.0,None,None,None,None,None,None,8390951.0
2,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1681324161,2023-04-12 18:29:21.000000,2022-01-01,2022-12-31,41400449.0,27047461.0,14352988.0,163597.0,None,...,None,None,10008065.0,None,None,None,None,None,None,10008065.0
3,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1681324161,2023-04-12 18:29:21.000000,2021-01-01,2021-12-31,41152800.0,26415762.0,14737038.0,-2861.0,None,...,None,None,10446634.0,None,None,None,None,None,None,10446634.0
4,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1712936573,2024-04-12 15:42:53.000000,2023-01-01,2023-12-31,46690658.0,27642181.0,19048477.0,600111.0,None,...,None,None,14630430.0,None,None,None,None,None,None,14630430.0
5,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1712936573,2024-04-12 15:42:53.000000,2022-01-01,2022-12-31,41400449.0,27047461.0,14352988.0,163597.0,None,...,None,None,10008065.0,None,None,None,None,None,None,10008065.0
6,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1745016653,2025-04-18 22:50:53.000000,2024-01-01,2024-12-31,52343305.0,27951556.0,24391749.0,866826.0,None,...,None,None,17978665.0,None,None,None,None,None,None,17978665.0
7,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1745016653,2025-04-18 22:50:53.000000,2023-01-01,2023-12-31,46690658.0,27642181.0,19048477.0,600111.0,None,...,None,None,14630430.0,None,None,None,None,None,None,14630430.0


## Extract Income Statement for Other Entities

This function can now be tested with additional entities to ensure it works with different entities. It now will be tested with Phillips 66 Carrier Pipeline and Phillips 66 Pipeline.

In [21]:
# Use str.contains to find pipelines with Phillips in the name
df_is[df_is.loc[:, 'filing_name'].str.contains('Phillips')]

,entity_id,filing_name,publication_time,start_date,end_date,provision_for_deferred_taxes_extraordinary_items,extraordinary_items_net,equity_in_earnings_losses_of_affiliated_companies_including_dividend_income,net_income_loss,income_taxes_on_extraordinary_items,...,federal_income_taxes_on_income_from_continuing_operations,net_carrier_operating_income,miscellaneous_income_charges,income_net_from_noncarrier_property,unusual_or_infrequent_items_debit,operating_revenues,other_income_and_deductions,interest_and_dividend_income,extraordinary_items,income_loss_before_extraordinary_items
328,C001136,"ConocoPhillips_Transportation_Alaska,_Inc._for...",2024-05-17 20:32:50.000000,2020-01-01,2020-12-31,NaN,NaN,234613.0,61233390.0,NaN,...,21197551.0,159834960.0,79743905.0,-341.0,NaN,393749698.0,-85681228.0,19315.0,NaN,61233390.0
329,C001136,"ConocoPhillips_Transportation_Alaska,_Inc._for...",2024-05-17 20:32:50.000000,2021-01-01,2021-12-31,NaN,NaN,-921047.0,38936315.0,NaN,...,13041132.0,144671096.0,81727228.0,NaN,NaN,385605145.0,-84813994.0,1206594.0,NaN,38936315.0
391,C001136,"ConocoPhillips_Transportation_Alaska,_Inc._for...",2022-04-05 12:46:14.000000,2020-01-01,2020-12-31,NaN,NaN,234613.0,61233390.0,NaN,...,21197551.0,159834960.0,79743905.0,-341.0,NaN,393749698.0,-85681228.0,19315.0,NaN,61233390.0
392,C001136,"ConocoPhillips_Transportation_Alaska,_Inc._for...",2022-04-05 12:46:14.000000,2021-01-01,2021-12-31,NaN,NaN,-921047.0,38936315.0,NaN,...,13041132.0,144947729.0,82003861.0,NaN,NaN,385605145.0,-85090627.0,1206594.0,NaN,38936315.0
496,C004907,Phillips_66_Sweeny-Freeport_LLC_form6_Q4_16503...,2022-04-18 19:48:06.000000,2021-01-01,2021-12-31,NaN,NaN,NaN,44566183.0,NaN,...,NaN,44469189.0,66902.0,NaN,NaN,83263279.0,96994.0,163896.0,NaN,44566183.0
497,C004907,Phillips_66_Sweeny-Freeport_LLC_form6_Q4_16503...,2022-04-18 19:48:06.000000,2020-01-01,2020-12-31,NaN,NaN,NaN,77963878.0,NaN,...,NaN,70077512.0,143.0,NaN,NaN,105646817.0,7886366.0,886509.0,NaN,77963878.0
558,C003494,Phillips_66_Carrier_LLC_form6_Q4_1650310849,2022-04-18 19:40:49.000000,2020-01-01,2020-12-31,NaN,NaN,NaN,210470630.0,NaN,...,NaN,159886578.0,28126.0,50176796.0,NaN,424828311.0,50584052.0,NaN,NaN,210470630.0
559,C003494,Phillips_66_Carrier_LLC_form6_Q4_1650310849,2022-04-18 19:40:49.000000,2021-01-01,2021-12-31,NaN,NaN,NaN,255646850.0,NaN,...,NaN,217916145.0,18494.0,36879551.0,NaN,486569876.0,37730705.0,NaN,NaN,255646850.0
598,C003013,Phillips_66_Pipeline_LLC_form6_Q4_1650309999,2022-04-18 19:26:39.000000,2021-01-01,2021-12-31,0.0,0.0,19839005.0,50909633.0,0.0,...,0.0,33052273.0,85785.0,-21850462.0,0.0,186956983.0,17857360.0,13351202.0,NaN,50909633.0
599,C003013,Phillips_66_Pipeline_LLC_form6_Q4_1650309999,2022-04-18 19:26:39.000000,2020-01-01,2020-12-31,0.0,0.0,19172693.0,15669169.0,0.0,...,0.0,-15431286.0,145339.0,-17245524.0,0.0,158754656.0,31100455.0,14151274.0,0.0,15669169.0


In [22]:
# Assign additional entity_id's to variables
id_p66_carrier = 'C003494'
id_p66_pieline = 'C003013'

In [23]:
# Extract additional income statements using entitiy_id's
df_p66_carrier = get_income_statement(db_file, sql_query, id_p66_carrier)
df_p66_pipeline = get_income_statement(db_file, sql_query, id_p66_pieline)

In [24]:
df_p66_carrier

,entity_id,filing_name,publication_time,start_date,end_date,operating_revenues,operating_expenses,net_carrier_operating_income,other_income_and_deductions,income_net_from_noncarrier_property,...,gain_loss_from_disposition_of_discontinued_segments_less_applicable_income_taxes,income_loss_from_discontinued_operations,income_loss_before_extraordinary_items,extraordinary_items_net,income_taxes_on_extraordinary_items,provision_for_deferred_taxes_extraordinary_items,extraordinary_items,cumulative_effect_of_changes_in_accounting_principles_less_applicable_income_taxes,extraordinary_items_and_accounting_changes,net_income_loss
0,C003494,Phillips_66_Carrier_LLC_form6_Q4_1650310849,2022-04-18 19:40:49.000000,2020-01-01,2020-12-31,424828311.0,264941733.0,159886578.0,50584052.0,50176796.0,...,None,None,210470630.0,None,None,None,None,None,None,210470630.0
1,C003494,Phillips_66_Carrier_LLC_form6_Q4_1650310849,2022-04-18 19:40:49.000000,2021-01-01,2021-12-31,486569876.0,268653731.0,217916145.0,37730705.0,36879551.0,...,None,None,255646850.0,None,None,None,None,None,None,255646850.0
2,C003494,Phillips_66_Carrier_LLC_form6_Q4_1681321057,2023-04-12 17:37:37.000000,2022-01-01,2022-12-31,569108563.0,276517762.0,292590801.0,37180136.0,33081352.0,...,None,None,329770937.0,None,None,None,None,None,None,329770937.0
3,C003494,Phillips_66_Carrier_LLC_form6_Q4_1681321057,2023-04-12 17:37:37.000000,2021-01-01,2021-12-31,486569876.0,268653731.0,217916145.0,37730705.0,36879551.0,...,None,None,255646850.0,None,None,None,None,None,None,255646850.0
4,C003494,Phillips_66_Carrier_LLC_form6_Q4_1712937587,2024-04-12 15:59:47.000000,2023-01-01,2023-12-31,631429929.0,273260536.0,358169393.0,58612084.0,30865131.0,...,None,None,416781477.0,None,None,None,None,None,None,416781477.0
5,C003494,Phillips_66_Carrier_LLC_form6_Q4_1712937587,2024-04-12 15:59:47.000000,2022-01-01,2022-12-31,569108563.0,276517762.0,292590801.0,37180136.0,33081352.0,...,None,None,329770937.0,None,None,None,None,None,None,329770937.0
6,C003494,Phillips_66_Carrier_LLC_form6_Q4_1745014218,2025-04-18 22:10:18.000000,2024-01-01,2024-12-31,672931827.0,266516602.0,406415225.0,90025106.0,47169288.0,...,None,None,496440331.0,None,None,None,None,None,None,496440331.0
7,C003494,Phillips_66_Carrier_LLC_form6_Q4_1745014218,2025-04-18 22:10:18.000000,2023-01-01,2023-12-31,631429929.0,273260536.0,358169393.0,58612084.0,30865131.0,...,None,None,416781477.0,None,None,None,None,None,None,416781477.0


In [25]:
df_p66_pipeline

,entity_id,filing_name,publication_time,start_date,end_date,operating_revenues,operating_expenses,net_carrier_operating_income,other_income_and_deductions,income_net_from_noncarrier_property,...,gain_loss_from_disposition_of_discontinued_segments_less_applicable_income_taxes,income_loss_from_discontinued_operations,income_loss_before_extraordinary_items,extraordinary_items_net,income_taxes_on_extraordinary_items,provision_for_deferred_taxes_extraordinary_items,extraordinary_items,cumulative_effect_of_changes_in_accounting_principles_less_applicable_income_taxes,extraordinary_items_and_accounting_changes,net_income_loss
0,C003013,Phillips_66_Pipeline_LLC_form6_Q4_1650309999,2022-04-18 19:26:39.000000,2021-01-01,2021-12-31,186956983.0,153904710.0,33052273.0,17857360.0,-21850462.0,...,0.0,None,50909633.0,0.0,0.0,0.0,NaN,0.0,NaN,50909633.0
1,C003013,Phillips_66_Pipeline_LLC_form6_Q4_1650309999,2022-04-18 19:26:39.000000,2020-01-01,2020-12-31,158754656.0,174185942.0,-15431286.0,31100455.0,-17245524.0,...,0.0,None,15669169.0,0.0,0.0,0.0,0.0,0.0,0.0,15669169.0
2,C003013,Phillips_66_Pipeline_LLC_form6_Q4_1681324041,2023-04-12 18:27:21.000000,2021-01-01,2021-12-31,186956983.0,153904710.0,33052273.0,17857360.0,-21850462.0,...,NaN,None,50909633.0,NaN,NaN,NaN,NaN,NaN,NaN,50909633.0
3,C003013,Phillips_66_Pipeline_LLC_form6_Q4_1681324041,2023-04-12 18:27:21.000000,2022-01-01,2022-12-31,174934355.0,137804508.0,37129847.0,23159344.0,-20599749.0,...,NaN,None,60289191.0,NaN,NaN,NaN,NaN,NaN,NaN,60289191.0
4,C003013,Phillips_66_Pipeline_LLC_form6_Q4_1712937909,2024-04-12 16:05:09.000000,2023-01-01,2023-12-31,143117134.0,107075727.0,36041407.0,32722043.0,-16432648.0,...,NaN,None,68763450.0,NaN,NaN,NaN,NaN,NaN,NaN,68763450.0
5,C003013,Phillips_66_Pipeline_LLC_form6_Q4_1712937909,2024-04-12 16:05:09.000000,2022-01-01,2022-12-31,174934355.0,137804508.0,37129847.0,23159344.0,-20599749.0,...,NaN,None,60289191.0,NaN,NaN,NaN,NaN,NaN,NaN,60289191.0
6,C003013,Phillips_66_Pipeline_LLC_form6_Q4_1745012446,2025-04-18 21:40:46.000000,2024-01-01,2024-12-31,95864006.0,75369658.0,20494348.0,11384496.0,-29767622.0,...,NaN,None,31878844.0,NaN,NaN,NaN,NaN,NaN,NaN,31878844.0
7,C003013,Phillips_66_Pipeline_LLC_form6_Q4_1745012446,2025-04-18 21:40:46.000000,2023-01-01,2023-12-31,143117134.0,107075727.0,36041407.0,32722043.0,-16432648.0,...,NaN,None,68763450.0,NaN,NaN,NaN,NaN,NaN,NaN,68763450.0


## Conclusion

The function does appear to work for additional pipelines. From here a similar approach could also be taken for other FERC Form 6 statements, such as the Balance Sheet. 